### **1. 리뷰데이터 크롤링하기 (최신순으로)**

In [144]:
import time
import pandas as pd
import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
import seaborn as sns


In [129]:
url = "https://play.google.com/store/apps/details?id=com.shinhan.sbanking&showAllReviews=true"  # 접속하고자하는 url
driverPath = "/Users/parksoomin/Desktop/chromedriver"  # driver file path
driver = webdriver.Chrome(driverPath)  # Open Chrome
driver.get(url)  # Enter the url

In [135]:
# 최신순으로 설정하기 
driver.find_element_by_xpath("//span[@class='DPvwYc']").click()
time.sleep(1)
driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/c-wiz/div[1]/div/div[2]/div[1]/span').click()

ElementClickInterceptedException: Message: element click intercepted: Element <span class="DPvwYc" aria-hidden="true">...</span> is not clickable at point (352, 12). Other element would receive the click: <a href="/store/apps" class="L9ZZW uJjCzb">...</a>
  (Session info: chrome=88.0.4324.146)


In [134]:

SCROLL_PAUSE_TIME = 0.5
num_review=0
while(num_review<400):
    # 1. 맨 아래까지 총 4번의 스크롤
    for i in range(1,5):
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      # Wait to load page
      time.sleep(SCROLL_PAUSE_TIME)
      now_reviews=driver.find_elements_by_xpath("//span[@jsname='bN97Pc']")
      num_review=len(now_reviews)
    # 2. 더보기 클릭
    try:
        # 더보기 버튼찾기
        element = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span')
        if (element is not None):
            element.click()
            print(num_review)
    except Exception:
        continue
    


In [122]:
# reviews = driver.find_elements_by_xpath("//span[@jsname='bN97Pc']")
# dates = driver.find_elements_by_xpath("//span[@class='p2TkOb']")
# likes = driver.find_elements_by_xpath("//div[@aria-label='이 리뷰가 유용하다는 평가를 받은 횟수입니다.']")
# stars = driver.find_elements_by_xpath("//span[@class='nt2C1d']/div[@class='pf5lIe']/div[@role='img']")



In [142]:
reviews = driver.find_elements_by_xpath("//span[@jsname='bN97Pc']")
dates = driver.find_elements_by_xpath("//span[@class='p2TkOb']")
likes = driver.find_elements_by_xpath("//div[@aria-label='이 리뷰가 유용하다는 평가를 받은 횟수입니다.']")
stars = driver.find_elements_by_xpath("//span[@class='nt2C1d']/div[@class='pf5lIe']/div[@role='img']")

data_list = []
for i in range(len(reviews)):
         newdate=dates[i].text.replace('년 ','').replace('월 ','').replace('일','')
         data_list.append({
         'DATE' : datetime.datetime.strptime(newdate,"%Y%m%d"),
         'STAR' : stars[i].get_attribute('aria-label'),
         'LIKE' : likes[i].text,
         'REVIEW_TEXT' : reviews[i].text })
data_df = pd.DataFrame(data_list)
print(data_df)


          DATE                  STAR LIKE  \
0   2021-12-05  별표 5개 만점에 1개를 받았습니다.   10   
1   2021-12-05  별표 5개 만점에 2개를 받았습니다.    2   
2   2021-12-05  별표 5개 만점에 1개를 받았습니다.    1   
3   2021-12-05  별표 5개 만점에 1개를 받았습니다.    1   
4   2021-12-06  별표 5개 만점에 1개를 받았습니다.    2   
..         ...                   ...  ...   
635 2020-12-25  별표 5개 만점에 1개를 받았습니다.        
636 2021-12-02  별표 5개 만점에 2개를 받았습니다.        
637 2020-11-21  별표 5개 만점에 3개를 받았습니다.        
638 2020-11-24  별표 5개 만점에 1개를 받았습니다.    1   
639 2021-12-05  별표 5개 만점에 1개를 받았습니다.   15   

                                           REVIEW_TEXT  
0    1시간이 남도록 서버에서 오류가 난다고 실행이 안됩니다. Pc로 이용하려해도 모바일...  
1    비정상적인 앱이라고 정상유통된 앱을 다시 다운받으라는 데 무슨 소리인지 모르겠습니다...  
2    LTE 쓰고있는데 무선통신(Wi-fi)문제로 처음부터 다시 실행하라고 함. 아무리 ...  
3    사흘전만해도 빠르게 사용이 가능했는데 갑자기 로그인창에서 다음화면으로 넘어가질 않아...  
4    신분증 인식이 너무 이상합니다. 제폰이 초점맞는데 시간이 걸려서 초점맞출려고 기다리...  
..                                                 ...  
635                       어플 계속 통신오류나고 화나고 짜증나서 못써먹겠네요  
636

In [148]:
data_df.to_csv('review(2.8).csv')